<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Data Science Para Análise Multivariada de Dados</font>
## <font color='blue'>Projeto 9</font>
### <font color='blue'>Data Science no Agronegócio</font>
### <font color='blue'>Previsão de Rendimento de Colheita e Otimização da Irrigação</font>

## Instalando e Carregando os Pacotes
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

https://www.tensorflow.org/

In [1]:
%pip install tensorflow
%env TF_CPP_MIN_LOG_LEVEL=3

Note: you may need to restart the kernel to use updated packages.Collecting tensorflow
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.12.23-py2.py3-none-any.whl.metadata (876 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.68.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.

In [2]:
# Imports
import joblib
import sklearn
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

## Carregando o Conjunto de Dados

In [3]:
# Carregar o dataset
df_dsa = pd.read_csv('dataset.csv')

In [4]:
df_dsa.shape

(124, 12)

In [5]:
df_dsa.head()

,data,indice_vegetacao,capacidade_solo,concentracao_co2,nivel_nutrientes,indice_fertilizantes,profundidade_raiz,radiacao_solar,precipitacao,estagio_crescimento,historico_rendimento,umidade
0,2012-12-01,323,455,3102.61,423.45,844.0,468.0,578.0,28.67,207.70504,117.7,79.261905
1,2013-01-01,345,546,3100.45,415.85,799.0,485.0,557.0,24.49,228.94287,4.5,82.193548
2,2013-02-01,362,595,3199.41,410.77,718.0,466.0,552.0,22.06,238.41747,25.1,74.839286
3,2013-03-01,376,636,3281.67,414.82,614.0,442.0,574.0,21.64,218.47599,53.6,77.935484
4,2013-04-01,383,738,3261.65,451.04,619.0,429.0,595.0,22.30,226.15010,166.0,80.450000


In [6]:
df_dsa.tail()

,data,indice_vegetacao,capacidade_solo,concentracao_co2,nivel_nutrientes,indice_fertilizantes,profundidade_raiz,radiacao_solar,precipitacao,estagio_crescimento,historico_rendimento,umidade
119,2022-11-01,362,363,2626.91,1252.78,738.07,427.49,1430.48,60.18,186.68326,38.2,77.950000
120,2022-12-01,310,322,2736.64,1287.68,749.57,385.09,1472.27,62.25,210.72987,33.7,76.177419
121,2023-01-01,277,307,2842.81,1289.12,761.60,373.03,1525.43,63.04,244.41912,4.6,74.774194
122,2023-02-01,323,330,2936.19,1303.59,759.59,390.69,1572.25,71.52,223.31732,6.9,66.910714
123,2023-03-01,360,339,2847.84,1234.88,771.62,396.87,1302.61,74.80,228.56676,41.5,69.000000


## Análise Exploratória

In [7]:
# Verificar os tipos de dados das colunas
df_dsa.dtypes

data                     object
indice_vegetacao          int64
capacidade_solo           int64
concentracao_co2        float64
nivel_nutrientes        float64
indice_fertilizantes    float64
profundidade_raiz       float64
radiacao_solar          float64
precipitacao            float64
estagio_crescimento     float64
historico_rendimento    float64
umidade                 float64
dtype: object

In [8]:
# Exibir as colunas do dataset
df_dsa.columns

Index(['data', 'indice_vegetacao', 'capacidade_solo', 'concentracao_co2',
       'nivel_nutrientes', 'indice_fertilizantes', 'profundidade_raiz',
       'radiacao_solar', 'precipitacao', 'estagio_crescimento',
       'historico_rendimento', 'umidade'],
      dtype='object')

In [9]:
# Identificar colunas não numéricas
non_numeric_columns = df_dsa.select_dtypes(include = ['object']).columns
print(f'Colunas não numéricas: {non_numeric_columns}')

Colunas não numéricas: Index(['data'], dtype='object')


## Limpeza e Transformação

In [10]:
# Remover colunas não numéricas (se não forem necessárias)
df_dsa = df_dsa.drop(columns = non_numeric_columns)

In [11]:
# Verificar se a coluna 'umidade' contém valores numéricos
if df_dsa['umidade'].dtype == 'object':
    df_dsa['umidade'] = pd.to_numeric(df_dsa['umidade'], errors = 'coerce')

In [12]:
# Remover linhas com valores faltantes
df_dsa = df_dsa.dropna()

## Padronização dos Dados

In [13]:
# Separar preditores e variável alvo
X = df_dsa.drop(columns = 'umidade')
y = df_dsa['umidade']

In [14]:
# Dividir os dados em conjunto de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
# Cria o padronizador
scaler = StandardScaler()

In [16]:
# Padroniza os dados
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

In [17]:
# Salva o padronizador no disco
joblib.dump(scaler, 'scaler_dsa.joblib')

['scaler_dsa.joblib']

## Construção do Modelo

Acompanhe os detalhes no videobook do Capítulo 19.

In [18]:
# Definir a arquitetura do modelo
modelo_dsa = Sequential([Dense(64, activation = 'relu', input_shape = (X_treino.shape[1],)),
                         Dropout(0.3),
                         Dense(32, activation = 'relu'),
                         Dropout(0.3),
                         Dense(16, activation = 'relu'),
                         Dense(1)])

O código acima define a arquitetura de um modelo de rede neural sequencial utilizando a biblioteca Keras. Aqui está uma explicação detalhada de cada linha:

**modelo_dsa = Sequential([ ... ])**: Cria um modelo sequencial, que é uma pilha linear de camadas.

**Dense(64, activation='relu', input_shape=(X_treino.shape[1],))**: Adiciona uma camada densa (totalmente conectada) com 64 neurônios e função de ativação ReLU (Rectified Linear Unit). A input_shape especifica a forma dos dados de entrada, que corresponde ao número de características (colunas) em X_treino.

**Dropout(0.3)**: Adiciona uma camada de dropout com uma taxa de 30%. Dropout é uma técnica de regularização que desativa aleatoriamente 30% dos neurônios durante o treinamento para prevenir o overfitting.

**Dense(32, activation='relu')**: Adiciona outra camada densa com 32 neurônios e função de ativação ReLU.

**Dropout(0.3)**: Adiciona mais uma camada de dropout com uma taxa de 30%.

**Dense(16, activation='relu')**: Adiciona mais uma camada densa com 16 neurônios e função de ativação ReLU.

**Dense(1)**: Adiciona a camada de saída com um único neurônio. Não é especificada uma função de ativação, o que é comum em problemas de regressão, onde a saída é um valor contínuo. Se fosse um problema de classificação binária, uma ativação sigmoid poderia ser usada nesta camada.

Essa arquitetura é típica para problemas de regressão onde se deseja prever um único valor contínuo a partir de múltiplas características de entrada.

## Compilação do Modelo

Acompanhe os detalhes no videobook do Capítulo 19.

In [19]:
# Compilar o modelo
modelo_dsa.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

O código acima compila o modelo de rede neural que foi definido anteriormente. A compilação é um passo necessário antes de treinar o modelo. Aqui está uma explicação de cada parte do código:

**modelo_dsa.compile(...)**: Compila o modelo, configurando-o para treinamento com um otimizador, uma função de perda e métricas.

**optimizer='adam'**: Define o otimizador como 'adam'. O Adam (Adaptive Moment Estimation) é um otimizador eficiente e amplamente utilizado em redes neurais, combinando as vantagens dos algoritmos de Gradient Descent com Momentum e RMSProp. Ele ajusta dinamicamente a taxa de aprendizado durante o treinamento.

**loss='mse'**: Define a função de perda como 'mse' (Mean Squared Error). MSE é uma função de perda comum em problemas de regressão, que calcula a média dos quadrados das diferenças entre os valores previstos e os valores reais. É usada para medir o desempenho do modelo, com valores menores indicando um melhor ajuste.

**metrics=['mae']**: Especifica que a métrica a ser monitorada durante o treinamento é 'mae' (Mean Absolute Error). MAE é a média dos valores absolutos das diferenças entre os valores previstos e os valores reais. Assim como MSE, valores menores de MAE indicam um melhor desempenho do modelo, mas MAE é menos sensível a grandes erros do que MSE.

Compilar o modelo é um passo essencial que define como o modelo será otimizado e avaliado durante o treinamento.

## Definindo os Callbacks

Acompanhe os detalhes no videobook do Capítulo 19.

In [20]:
# Callbacks
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)
model_checkpoint = ModelCheckpoint('modelo_dsa.keras', save_best_only = True)

Este trecho de código define dois callbacks para o treinamento do modelo: EarlyStopping e ModelCheckpoint. Callbacks são ferramentas que permitem realizar certas ações em pontos específicos durante o treinamento.

**early_stopping = EarlyStopping(...)**: Define um callback de parada antecipada (early stopping).

**monitor='val_loss'**: Monitora a perda de validação (val_loss) durante o treinamento. A validação é um processo onde um conjunto separado de dados (dados de validação) é usado para avaliar o desempenho do modelo, ajudando a evitar o overfitting.

**patience=10**: Define a paciência em 10 épocas. Isso significa que se a perda de validação não melhorar por 10 épocas consecutivas, o treinamento será interrompido.

**restore_best_weights=True**: Restabelece os pesos do modelo para o estado em que apresentaram a melhor perda de validação. Isso garante que os pesos finais do modelo sejam os melhores encontrados durante o treinamento.

**model_checkpoint = ModelCheckpoint(...)**: Define um callback de checkpoint do modelo.

**'modelo_dsa.keras'**: Especifica o nome do arquivo onde o modelo será salvo.

**save_best_only=True**: Salva o modelo apenas quando ele apresentar a melhor perda de validação até o momento. Isso evita salvar várias versões do modelo e garante que o melhor modelo encontrado durante o treinamento seja salvo.

Em resumo, esses callbacks ajudam a:

- Parar o treinamento antecipadamente se o modelo não estiver melhorando, economizando tempo e recursos computacionais.

- Salvar automaticamente o melhor modelo encontrado durante o treinamento, garantindo que você tenha uma versão do modelo com o melhor desempenho.

## Treinamento do Modelo

In [21]:
modelo_dsa.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,329 (13.00 KB)

 Trainable params: 3,329 (13.00 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Treinar o modelo
history = modelo_dsa.fit(X_treino_scaled, 
                         y_treino,
                         validation_split = 0.2,
                         epochs = 100,
                         batch_size = 32,
                         callbacks = [early_stopping, model_checkpoint])

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 651ms/step - loss: 5181.2212 - mae: 71.8467 - val_loss: 5177.7759 - val_mae: 71.8520
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 5166.7476 - mae: 71.7309 - val_loss: 5154.3896 - val_mae: 71.6893
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 5130.8916 - mae: 71.5060 - val_loss: 5127.9751 - val_mae: 71.5050
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 5099.7959 - mae: 71.2689 - val_loss: 5098.3296 - val_mae: 71.2973
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 5050.4004 - mae: 70.9180 - val_loss: 5064.4434 - val_mae: 71.0588
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 5020.9785 - mae: 70.7094 - val_loss: 5025.4229 - val_mae: 70.7832
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 4980.2710 - mae: 70.4227 - val_loss: 4979.8374 - val_mae: 70.4602
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 4935.5059 - mae: 70.1016 - val_loss: 4928.2529 - val_mae: 70.0930


## Avaliação do Modelo

In [23]:
# Avaliar o modelo no conjunto de teste
teste_loss, teste_mae = modelo_dsa.evaluate(X_teste_scaled, y_teste)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 174.7564 - mae: 11.6600


In [24]:
print(f'Teste Loss: {teste_loss}')
print(f'Teste MAE: {teste_mae}')

Teste Loss: 174.75640869140625
Teste MAE: 11.660011291503906


Vamos agora fazer o deploy do modelo via API no próximo capítulo.

# Fim